In [5]:
# =============================================================================
# PROJECT 3: Healthcare Readmission Analysis with SDoH Data
# =============================================================================
# Scenario: Integrating internal patient data with external public CSV data.
# Skills: ETL, Data Blending, Geospatial Analysis Concepts, Statistical
#         Correlation, SQL, Pandas, Data Visualization (Plotly).
# =============================================================================

# Step 0: Setup and Data Simulation
# -----------------------------------------------------------------------------
import pandas as pd
import numpy as np
import io
import sqlite3
import plotly.express as px
import requests
import zipfile
import os

# --- Source 1: Internal Patient Data Export (Simulated) ---
# We'll use a well-known public dataset as a proxy for an internal export.
def get_patient_data_csv():
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip'
    local_zip_path = 'dataset_diabetes.zip'
    # extracted_csv_name = 'diabetic_data.csv' # This might be incorrect

    # Download the zip file
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes
    with open(local_zip_path, 'wb') as f:
        f.write(response.content)

    # Inspect the contents of the zip file
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        print("Contents of the zip file:")
        for file_info in zip_ref.infolist():
            print(file_info.filename)
        # Assuming the desired CSV is the one with 'data' in the name and ends with '.csv'
        csv_files = [f.filename for f in zip_ref.infolist() if 'data' in f.filename and f.filename.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No suitable CSV file found in the zip archive.")
        extracted_csv_name = csv_files[0] # Take the first matching file

        # Extract the specific CSV file from the zip
        zip_ref.extract(extracted_csv_name)


    # Read the extracted CSV into a pandas DataFrame
    df = pd.read_csv(extracted_csv_name, sep=',')

    # Clean up the downloaded zip and extracted csv
    os.remove(local_zip_path)
    os.remove(extracted_csv_name)

    # Simulate adding a hospital location ZIP code to each encounter
    np.random.seed(42)
    hospital_zips = [90210, 60601, 33101, 10001, 77001]
    df['hospital_zip_code'] = np.random.choice(hospital_zips, size=len(df))
    return df.to_csv(index=False)

# --- Source 2: Public SDoH Data Export (CSV) ---
def get_sdoh_data_csv():
    return """zip_code,county_name,median_income,poverty_rate,percent_uninsured
90210,Los Angeles County,110000,8.5,5.2
60601,Cook County,95000,15.2,10.1
33101,Miami-Dade County,45000,25.8,22.5
10001,New York County,85000,18.1,12.3
77001,Harris County,55000,22.4,20.1
"""

# --- Load data from simulated CSVs ---
df_patients = pd.read_csv(io.StringIO(get_patient_data_csv()))
df_sdoh = pd.read_csv(io.StringIO(get_sdoh_data_csv()))

print("--- Data sources loaded successfully ---")


# Step 1: ETL - Cleaning and Data Blending
# -----------------------------------------------------------------------------
print("\n--- Step 1: Cleaning and Blending Data ---")

# Clean patient data: Handle missing values and create a clear target variable
df_patients.replace('?', pd.NA, inplace=True)
relevant_cols = [
    'race', 'gender', 'age', 'time_in_hospital',
    'readmitted', 'hospital_zip_code'
]
df_clean = df_patients[relevant_cols].copy().dropna()

# We are interested in early readmissions (<30 days) as a key performance indicator
df_clean['is_readmitted_early'] = (df_clean['readmitted'] == '<30').astype(int)

# --- The Core Blending Step ---
# Join the internal patient data with the external public SDoH data.
df_enriched = pd.merge(
    df_clean,
    df_sdoh,
    left_on='hospital_zip_code',
    right_on='zip_code',
    how='left'
)

print("Internal and external data blended successfully.")
print(df_enriched[['age', 'is_readmitted_early', 'hospital_zip_code', 'median_income', 'poverty_rate']].head())


# Step 2: Aggregation and Analysis
# -----------------------------------------------------------------------------
print("\n--- Step 2: Aggregating Data for Analysis ---")

# Aggregate the blended data by geography (ZIP code) to analyze community-level trends.
# This is a typical operation done in SQL in a real data warehouse.
df_geo_analysis = df_enriched.groupby('hospital_zip_code').agg(
    ReadmissionRate=('is_readmitted_early', lambda x: x.mean() * 100),
    AvgPovertyRate=('poverty_rate', 'mean'),
    AvgMedianIncome=('median_income', 'mean'),
    TotalEncounters=('age', 'count')
).reset_index()

# Join back to get the county name for better labeling
df_geo_analysis = pd.merge(df_geo_analysis, df_sdoh[['zip_code', 'county_name']],
                           left_on='hospital_zip_code', right_on='zip_code', how='left').drop('zip_code', axis=1)


print("Aggregated analysis by geography:")
print(df_geo_analysis)


# Step 3: Loading to Warehouse and Visualization
# -----------------------------------------------------------------------------
print("\n--- Step 3: Visualizing Correlation and Geographic Patterns ---")

# --- Load to a simulated Data Warehouse (SQLite) ---
conn = sqlite3.connect('healthcare_warehouse.db')
df_enriched.to_sql('EnrichedPatientEncounters', conn, if_exists='replace', index=False)
df_geo_analysis.to_sql('GeographicAnalysis', conn, if_exists='replace', index=False)
conn.close()

# --- Correlation Scatter Plot ---
# Is there a relationship between poverty and readmission rates?
fig_corr = px.scatter(
    df_geo_analysis,
    x="AvgPovertyRate",
    y="ReadmissionRate",
    size="TotalEncounters",
    color="AvgMedianIncome",
    hover_name="county_name",
    trendline="ols",  # Ordinary Least Squares regression line shows the trend
    title="Community Poverty Rate vs. Hospital Readmission Rate",
    labels={
        "AvgPovertyRate": "Average Poverty Rate in Hospital's ZIP Code (%)",
        "ReadmissionRate": "Early Readmission Rate (%)",
        "AvgMedianIncome": "Avg. Median Income"
    },
    template='plotly_white'
)
fig_corr.update_layout(title_x=0.5)
fig_corr.show()

# --- Geographic 'Hotspot' Bar Chart ---
# A bar chart is a great way to show geographic disparities without needing complex map files.
fig_bar = px.bar(
    df_geo_analysis.sort_values('ReadmissionRate', ascending=False),
    x='county_name',
    y='ReadmissionRate',
    color='ReadmissionRate',
    color_continuous_scale=px.colors.sequential.Reds,
    title='Readmission Rate "Hotspots" by Hospital Location',
    labels={'county_name': 'Hospital County', 'ReadmissionRate': 'Early Readmission Rate (%)'},
    template='plotly_white'
)
fig_bar.update_layout(title_x=0.5)
fig_bar.show()

Contents of the zip file:
dataset_diabetes/diabetic_data.csv
dataset_diabetes/IDs_mapping.csv
--- Data sources loaded successfully ---

--- Step 1: Cleaning and Blending Data ---
Internal and external data blended successfully.
       age  is_readmitted_early  hospital_zip_code  median_income  \
0   [0-10)                    0              10001          85000   
1  [10-20)                    0              77001          55000   
2  [20-30)                    0              33101          45000   
3  [30-40)                    0              77001          55000   
4  [40-50)                    0              77001          55000   

   poverty_rate  
0          18.1  
1          22.4  
2          25.8  
3          22.4  
4          22.4  

--- Step 2: Aggregating Data for Analysis ---
Aggregated analysis by geography:
   hospital_zip_code  ReadmissionRate  AvgPovertyRate  AvgMedianIncome  \
0              10001        11.051389            18.1          85000.0   
1              33101